In [1]:
import pandas as pd
import dill
import re

classifier = dill.load(open('model.sav', 'rb'))
tfvectorizer = dill.load(open('tfidf.sav', 'rb'))
vectorizer = dill.load(open('bow.sav', 'rb'))

In [2]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
def clean(s):
    s=str(s)
    s=s.lower()
    html=re.compile('<.*?>')   #removing html tags
    cleaned = re.sub(html,' ',s)
    fil=[]
    for i in cleaned.split():
        if i!='c++':
            cleaned=re.sub('[^A-Za-z]+', '', i)
            fil.append(cleaned)
        else:
            fil.append(i)
    return fil
stop=set(stopwords.words('english'))
sno=SnowballStemmer('english')
def stem(s):
    fil=[]
    for _ in s:
        if _ not in stop:
            s=(sno.stem(_).encode('utf8'))
            fil.append(s)
    s=b' '.join(fil)
    return s

In [7]:
from tkinter import *
from PIL import ImageTk, Image
import os
root = Tk()

root.title("Autonomus Tagging Of Stackoverflow Questions")


img = ImageTk.PhotoImage(Image.open("index.png"))
panel = Label(root, image = img)
panel.image = img
panel.grid(row = 0, column = 0)
#panel.pack()

img1 = ImageTk.PhotoImage(Image.open("st.jpg"))

label1 = Label(root,image = img1)
label1.image = img1
label1.grid(row = 1, column = 0)
#label1.pack()
label2 = Label(root, text = "Ask Me a Question!!!",font = "Arial 20 bold", fg = 'black')
label2.grid(row = 2, column = 0)
#label2.pack()

ques = StringVar()
quesEntered = Entry(root, width = 40, textvariable = ques,font="Arial 18 bold")
quesEntered.grid(column = 0, row = 3, padx = 3, pady = 3)
#quesEntered.pack()

def pred():
    t = ques.get()
    l=[]
    l.append(stem(clean(t)))
    x=tfvectorizer.transform(l)
    t=classifier.predict(x)
    k=vectorizer.inverse_transform(t)
    label3 = Label(root,text = re.sub('[^A-Za-z#+-]+', ' ', str(k[0])), font = "Arial 20 bold")
    label3.grid(column =0, row = 5, padx = 3, pady = 3)
    #label3.pack()

    
button = Button(root,text = "Predict Tag", font = "Arial 18 bold", command = pred)
button.grid(column= 0, row = 4, padx = 3, pady = 3)
#button.pack()


root.mainloop()
